In [ ]:
%run ../config/init.py

### Creating results folders
{% if cookiecutter.ngs_data_type == 'RNA-Seq' %}

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'alignments')
result_dir = os.path.join(RESULTS, DATASET, 'quantification')
if not os.path.exists(result_dir):
    os.mkdir(result_dir) 
os.chdir(result_dir)
{% if cookiecutter.sequencing_technology == 'paired-end' %}
samples = [ f.replace('_sorted.bam', '') for ds,dr,fs in os.walk(data_dir) for f in fs if f.endswith('.bam')]
{% else %}
samples = [ f.replace('_sorted.bam', '') for ds,dr,fs in os.walk(data_dir) for f in fs if f.endswith('.bam')]
{% endif %}

### Processing samples

In [ ]:
{% if cookiecutter.sequencing_technology == 'paired-end' %}
cmd_header = '{0} {1}/RNA-Seq/rnaseq-tpmcalculator.cwl -p --gtf {2} -q 255 -r {3} --genome_name {4} --sorted_bam'.format(
        CWLRUNNER, CWLWORKFLOWS, GENOME_GTF, GENOME_BED, GENOME_NAME)

{% else %}
cmd_header = '{0} {1}/RNA-Seq/rnaseq-tpmcalculator.cwl --gtf {2} -q 255 -r {3} --genome_name {4} --sorted_bam'.format(
        CWLRUNNER, CWLWORKFLOWS, GENOME_GTF, GENOME_BED, GENOME_NAME)
{% endif %}
log_suffix = 'quantification.log'
with open('commands', "w") as fin:
    for s in samples:
        r = os.path.join(data_dir, s + '_sorted.bam')
        if not os.path.exists(s + '_sorted_genes.out.gz'):
            fin.write('{0} {1} > {2}_{3} 2>&1\n'.format(cmd_header, r, s, log_suffix))

{% if cookiecutter.use_gnu_parallel == 'y' %}
!cat commands | parallel -j {{ cookiecutter.max_number_threads }}
{% else %}
!sh commands    
{% endif %}
check_errors_from_logs(result_dir, log_suffix)

### Creates TPM and reads matrices for genes

In [ ]:
data = {}
columns = ['ExonTPM', 'ExonReads']
output_suffix = "_sorted_genes.out"
files = [ f for ds, df, files in os.walk('./') for f in files if output_suffix in f]
for column in columns:
    print(column)
    data[column] = pandas.DataFrame()
    for f in files:
        # Get sample name removing the suffix and check if the output is compressed
        if f.endswith('.gz'):
            output_suffix_real = output_suffix + '.gz'
        else:
            output_suffix_real = output_suffix
        s = f.replace(output_suffix_real, '')
        df = pandas.read_csv(f, sep='\t')
        df = df[['Gene_Id', 'Chr', 'Start', 'End', 'ExonLength', column]]
        df = df.rename(index=str, columns={column: s})
        if data[column].empty:
            data[column] = df
        else:
            data[column] = data[column].merge(df, on=['Gene_Id', 'Chr', 'Start', 'End', 'ExonLength'], how='outer')
    print('Data columns: ' + str(len(data[column].columns)))
    print('Data rows: ' + str(len(data[column])))
    
    # Printing TSV matrices    
    data[column]['Gene_Chr_Start'] = data[column]['Gene_Id'] + '_' + data[column]["Chr"] + '_' + data[column]["Start"].map(str)
    data[column] = data[column].drop(['Gene_Id'], axis=1)
    cols = data[column].columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data[column] = data[column][cols]
    data[column].to_csv( column + '.tsv', sep='\t', index=False, na_rep='0')

### Plotting Exon TPM and read count per sample

In [ ]:
columns = ['ExonTPM', 'ExonReads']
output_suffix = "_sorted_genes.out"
files = [ f for ds, df, files in os.walk('./') for f in files if output_suffix in f]
files.sort()
for column in columns:    
    plt.figure(figsize=(10, 12)) 
    toPlot = []
    for f in files:
        if f.endswith('.gz'):
            output_suffix_real = output_suffix + '.gz'
        else:
            output_suffix_real = output_suffix
        s = f.replace(output_suffix_real, '')
        if s in data[column]:
            for r in data[column][s]:
                toPlot.append([r, s])
    d = pandas.DataFrame(toPlot, columns=[column, 'Sample'])
    ax = sns.boxplot(y='Sample', x=column, data=d, orient="h", palette="Set2")
{% elif cookiecutter.ngs_data_type == 'ChIP-Seq' %}

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'alignments')
result_dir = os.path.join(RESULTS, DATASET, 'peak-calling')
if not os.path.exists(result_dir):
    os.mkdir(result_dir) 
os.chdir(result_dir)
samples = [ f.replace('_sorted.tagAlign.gz', '') for ds,dr,fs in os.walk(data_dir) for f in fs if f.endswith('_sorted.tagAlign.gz')]

### Processing samples

In [ ]:
log_suffix = 'peak_calling.log'
cmd_header = '{0} {1}/ChIP-Seq/peak-calling-MACS2.cwl --homer_genome {2}9 --genome_fasta {3} --genome_gtf {4} --macs_callpeaks_g {5} --macs_callpeaks_q {6}'.format(
        CWLRUNNER, CWLWORKFLOWS, GENOME_MAPPABLE_SIZE, GENOME_FASTA, GENOME_GTF, 
        GENOME_MAPPABLE_SIZE, fdr)

with open('commands_peak_calling', "w") as fin:
    for s in factors['SampleID']:
        t = os.path.join(data_dir, s + '_sorted.tagAlign.gz')
        fin.write('{0} --tagAlign_gz {1} > {2}_{3} 2>&1\n'.format(cmd_header, t, s, log_suffix))
    for c in factors['condition'].unique():
        f = os.path.join(data_dir, c + '_R0.tagAlign.gz')
        if not os.path.exists(c + '_R0') and os.path.exists(f):            
            fin.write('{0} --tagAlign_gz {1} > {2}_{3} 2>&1\n'.format(cmd_header, f, c, log_suffix))

!cat commands_peak_calling | parallel -j {{ cookiecutter.max_number_threads }}

check_errors_from_logs(result_dir, log_suffix)

###  Irreproducible Discovery Rate (IDR)    

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'peak-calling')
result_dir = os.path.join(RESULTS, DATASET, 'idr')
if not os.path.exists(result_dir):
    os.mkdir(result_dir) 
os.chdir(result_dir)   

### Processing samples

In [ ]:
log_suffix = 'idr.log'
cmd_header = '{0} {1}/ChIP-Seq/idr.cwl --input_file_type narrowPeak --soft_idr_threshold 0.05 --homer_genome {2} --genome_fasta {3} --genome_gtf {4} '.format(
        CWLRUNNER, CWLWORKFLOWS, GENOME_NAME, GENOME_FASTA, GENOME_GTF,)

with open('commands_idr', "w") as fin:
    for c in factors['condition'].unique():
        peak_list = os.path.join(data_dir, c + '_R0_peaks.narrowPeak')
        if os.path.exists(peak_list):
            peak_list = '--pooled_peak_list {0}'.format(peak_list)
        else:
            peak_list = ''
        ids = factors[factors['condition'] == c]['SampleID']
        s_option = ''
        for s in ids:
            s = os.path.join(data_dir, s + '_sorted_peaks.narrowPeak')
            s_option = '{0} --samples {1}'.format(s_option, s)
        fin.write('{0} {1} {2} --output_file {3}.narrowPeak > {3}_{4} 2>&1\n'.format(cmd_header, s_option, peak_list, c, log_suffix))

!cat commands_idr | parallel -j {{ cookiecutter.max_number_threads }}

check_errors_from_logs(result_dir, log_suffix) 

{% elif cookiecutter.ngs_data_type == 'ChIP-exo' %} 

In [ ]:
factors = pandas.read_csv(os.path.join(DATA, DATASET, 'factors.txt'), sep='\t')
data_dir = os.path.join(RESULTS, DATASET, 'alignments')
result_dir = os.path.join(RESULTS, DATASET, 'peak_calling')
if not os.path.exists(result_dir):
    os.mkdir(result_dir) 
os.chdir(result_dir)

### Peak calling workflow
{% if cookiecutter.sequencing_technology == 'paired-end' %}
## No implemented yet. 

Please contact: veraalva@ncbi.nlm.nih.gov 
{% else %}    

In [ ]:
log_suffix = 'peak_calling.log'
cmd_header = '{0} {1}/ChIP-exo/peak-caller-MACE-SE.cwl --genome_gtf {2} --chrom_size {3} --tss_size 1000 '.format(
    CWLRUNNER, CWLWORKFLOWS,GENOME_GTF, GENOME_CHROMSIZES)

with open('commands_peak_calling', "w") as fin:
    for c in factors['condition'].unique():
        rep = factors[factors['condition'] == c]['SampleID']
        replicates = '{0} --output_basename {1}'.format(cmd_header, c)
        for r in rep:
            r = os.path.join(data_dir, r + '_sorted.bam')
            replicates += ' --sorted_bam {0}'.format(r)
        fin.write('{0} > {1}_{2} 2>&1\n'.format(replicates, c, log_suffix))
 

!cat commands_peak_calling | parallel -j {{ cookiecutter.max_number_threads }}

check_errors_from_logs(result_dir, log_suffix)
{% endif %}       
{% endif %}